In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import functools
import os.path as osp
from typing import List

import cv2
import gin
import mediapy as media
import numpy as np

from dycheck import core
from dycheck.datasets import Record3DProcessor
from dycheck.utils import annotation, common, image, io

gin.enter_interactive_mode()

In [ ]:
GIN_CONFIGS = ["../configs/iphone/process_record3d_to_iphone.gin"]
GIN_BINDINGS = [
    'DATA_ROOT="/home/hangg/datasets/iphone/record3d"',
    'SEQUENCE="paper-windmill"',
    # Dummy prompts since they will not be used.
    "FRGD_PROMPTS=['']",
    "Record3DProcessor.suppress_bad_frames_validation=True",
]

In [ ]:
with gin.unlock_config():
    core.parse_config_files_and_bindings(
        config_files=GIN_CONFIGS,
        bindings=GIN_BINDINGS,
        skip_unknown=True,
        master=False,
    )
config_str = gin.config_str()
print(f"*** Configuration:\n{config_str}")

In [ ]:
processor = Record3DProcessor()
processor._process_val_rgbas()

In [ ]:
val_bad_frames = []
for rgbas in processor.val_rgbas:
    # Downsample by 4 such that the image widget loads faster.
    bad_frames = annotation.annotate_record3d_bad_frames(
        common.parallel_map(
            lambda img: image.rescale(img, 1 / 4),
            list(rgbas[..., :3]),
        )
    )
    val_bad_frames.append(bad_frames)

In [ ]:
for i, bad_frames in enumerate(val_bad_frames):
    io.dump(
        osp.join(
            processor.data_dir,
            "Test",
            str(i),
            "RGBD_Video",
            "timesync_bad_frames.txt",
        ),
        bad_frames,
    )